In [1]:
import sqlalchemy as sa
import pandas as pd
import pyarrow as pa
import credentials_db as creds

In [2]:
pd.options.mode.copy_on_write = True

In [3]:
string_pa = pd.ArrowDtype(pa.string())

In [4]:
engine_sda = sa.create_engine('oracle+cx_oracle://'+creds.sda['usr']+':'+creds.sda['pwd']+creds.sda['host']+'/'+creds.sda['db'])

In [5]:
retention_rate_freshmen = .800
current_base_term = '202470'

In [6]:
txt_for_sql = open('./sql/enrollment data fall census.sql','r').read()

In [6]:
queries = []
for i in range (19,25):
    queries.append(txt_for_sql.replace('__yr__',str(i)))
sql_enrolllment_data_fall = '\nunion\n'.join(queries)

In [7]:
enrollment = pd.read_sql(sql_enrolllment_data_fall,engine_sda,dtype={
        'sytdean_pidm':'uint32[pyarrow]',
        'sytdean_term_code_key':'category',
        'sytdean_registered_ind':'category',
        'sytdean_camp_code':'category',
        'sytdean_levl_code':'category',
        'sytdean_coll_code_1':'category',
        'sytdean_majr_code_1':'category',
        'sytdean_styp_code':'category',
        'sytdean_class_code':'category'
    })

In [8]:
def retention(df: pd.DataFrame) -> pd.DataFrame:
    def retention_rate(s: pd.Series) -> float:
        return (s=='Y').mean()
    return (df
        .loc[df.sytdean_camp_code != 'S']
        .assign(
            classification = df.sytdean_styp_code.case_when(caselist = [
                (df.sytdean_styp_code.isin(['F','T']) & (df.sytdean_majr_code_1 == 'PHA6'), df.sytdean_styp_code.astype(string_pa) + '_PHA6'),
                (df.sytdean_styp_code.isin(['F','T']) & (df.sytdean_majr_code_1 == 'NURS'), df.sytdean_styp_code.astype(string_pa) + '_BSN'),
                (df.sytdean_styp_code.isin(['F','T']), df.sytdean_styp_code),
                (df.sytdean_styp_code.isin(['C','I','R']) & (df.sytdean_majr_code_1 == 'PHA6'), df.sytdean_class_code.astype(string_pa) + '_PHA6'),
                (df.sytdean_styp_code.isin(['C','I','R']) & (df.sytdean_majr_code_1 == 'NURS'), df.sytdean_class_code.astype(string_pa) + '_BSN'),
                (df.sytdean_styp_code.isin(['C','I','R']), df.sytdean_class_code.astype(string_pa)),
                (df.sytdean_styp_code.isin(['N','S','V']), 'NSV')
            ]).astype('category'),
            retention_term = (df.sytdean_term_code_key.astype('int64[pyarrow]')+100).astype(string_pa).astype('category')
        )
        .merge(df,how='left',left_on=['sytdean_pidm','retention_term'],right_on=['sytdean_pidm','sytdean_term_code_key'])
        .pivot_table(index='classification', columns='retention_term', values='sytdean_registered_ind_y', aggfunc=retention_rate, dropna=False, observed=False)
        .loc[:,'202070':'202470']
        .pipe(lambda df_: df_.assign(
            avg_5yr = df_.mean(axis=1),
            avg_3yr = df_.loc[:,'202270':'202470'].mean(axis=1)
        ))
    )

In [9]:
def add_ons(df: pd.DataFrame) -> pd.Series:
    return (df
        .loc[~df.sytdean_styp_code.isin(['F','T']) & (df.sytdean_camp_code != 'S') & (df.sytdean_term_code_key != '201970')]
        .assign(look_back_term = (df.sytdean_term_code_key.astype('int64[pyarrow]')-100).astype(string_pa).astype('category'))
        .merge(df,how='left',left_on=['sytdean_pidm','look_back_term'],right_on=['sytdean_pidm','sytdean_term_code_key'])
        .pipe(lambda df_: df_.loc[df_.sytdean_registered_ind_y !='Y'])
        .sytdean_term_code_key_x
        .value_counts()
        .sort_index()
        .loc['202070':]
        .pipe(lambda s_: pd.concat([s_, pd.Series(s_.mean(),index=['avg_5yr'])]))
        .pipe(lambda s_: pd.concat([s_, pd.Series(s_.loc['202270':'202470'].mean(),index=['avg_3yr'])]))
    )

In [11]:
def current_base(df: pd.DataFrame) -> pd.Series:
    return(df
        .loc[df.sytdean_term_code_key == current_base_term]
        .assign(classification = df.sytdean_styp_code.case_when(caselist = [
                (df.sytdean_styp_code.isin(['F','T']) & (df.sytdean_majr_code_1 == 'PHA6'), df.sytdean_styp_code.astype(string_pa) + '_PHA6'),
                (df.sytdean_styp_code.isin(['F','T']) & (df.sytdean_majr_code_1 == 'NURS'), df.sytdean_styp_code.astype(string_pa) + '_BSN'),
                (df.sytdean_styp_code.isin(['F','T']), df.sytdean_styp_code),
                (df.sytdean_styp_code.isin(['C','I','R']) & (df.sytdean_majr_code_1 == 'PHA6'), df.sytdean_class_code.astype(string_pa) + '_PHA6'),
                (df.sytdean_styp_code.isin(['C','I','R']) & (df.sytdean_majr_code_1 == 'NURS'), df.sytdean_class_code.astype(string_pa) + '_BSN'),
                (df.sytdean_styp_code.isin(['C','I','R']), df.sytdean_class_code.astype(string_pa)),
                (df.sytdean_styp_code.isin(['N','S','V']), 'NSV')
            ]).astype('category'))
        .classification
        .value_counts()
    )

In [12]:
retention_trends = retention(enrollment)
retention_trends

retention_term,202070,202170,202270,202370,202470,avg_5yr,avg_3yr
classification,,,,,,,
1Y,0.683962,0.631579,0.611511,0.645161,0.663755,0.647194,0.640143
1Y_PHA6,0.818182,0.428571,1.000000,0.777778,1.000000,0.804906,0.925926
2Y,0.877218,0.873227,0.866019,0.893006,0.878466,0.877587,0.879164
2Y_BSN,NaN,NaN,NaN,NaN,0.866667,0.866667,0.866667
2Y_PHA6,0.977273,0.971074,0.890110,0.944134,0.930636,0.942645,0.921627
3Y,0.808493,0.792905,0.736926,0.668349,0.744154,0.750165,0.716476
3Y_BSN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000
3Y_PHA6,0.990099,0.977778,0.957265,0.960265,0.972067,0.971495,0.963199
4Y,0.048631,0.047445,0.055098,0.051605,0.040472,0.048650,0.049058


In [13]:
add_ons(enrollment)

202070     221.0
202170     262.0
202270     283.0
202370     261.0
202470     230.0
avg_5yr    251.4
avg_3yr    258.0
dtype: float64